In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pykrige.ok import OrdinaryKriging
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
# Load the CSV file
file_path = 'ASJC.csv'  # Ensure your CSV is in the same folder or adjust the path
df = pd.read_csv(C:\Users\asadh\Downloads\Kriging\ASJC.csv)

In [ ]:
# Extract relevant columns
longitude = df['LongitudeDD'].values
latitude = df['LatitudeDD'].values
arsenic = df['As'].values

In [ ]:
# ------------------------------
# 1. Perform Ordinary Kriging
# ------------------------------
# Define the grid for prediction
gridx = np.linspace(min(longitude), max(longitude), 100)
gridy = np.linspace(min(latitude), max(latitude), 100)

In [ ]:
# Create Ordinary Kriging model with a spherical variogram
OK = OrdinaryKriging(longitude, latitude, arsenic, variogram_model='spherical', verbose=True)

In [ ]:
# Perform Kriging on the grid
arsenic_kriging, arsenic_std = OK.execute('grid', gridx, gridy)

In [ ]:
# Plot Kriging Prediction
plt.figure(figsize=(10, 6))
plt.contourf(gridx, gridy, arsenic_kriging, cmap='viridis')
plt.colorbar(label='Kriging Prediction (As)')
plt.scatter(longitude, latitude, c='white', edgecolor='black', label='Data Points')
plt.title('Kriging Prediction of Arsenic Concentration')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.show()

In [ ]:
# Plot Kriging Standard Error
plt.figure(figsize=(10, 6))
plt.contourf(gridx, gridy, arsenic_std, cmap='plasma')
plt.colorbar(label='Standard Error')
plt.scatter(longitude, latitude, c='white', edgecolor='black', label='Data Points')
plt.title('Kriging Standard Error')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.show()

In [ ]:
# ------------------------------
# 2. Fit a Global Linear Plane
# ------------------------------
# Prepare data for linear regression
coords = np.vstack((longitude, latitude)).T

In [ ]:
# Fit the linear regression model
linear_model = LinearRegression()
linear_model.fit(coords, arsenic)

In [ ]:
# Predict arsenic concentrations on the grid
grid_lon, grid_lat = np.meshgrid(gridx, gridy)
grid_coords = np.c_[grid_lon.ravel(), grid_lat.ravel()]
arsenic_global_pred = linear_model.predict(grid_coords).reshape(grid_lon.shape)

In [ ]:
# Plot Global Linear Plane Prediction
plt.figure(figsize=(10, 6))
plt.contourf(gridx, gridy, arsenic_global_pred, cmap='viridis')
plt.colorbar(label='Global Linear Plane Prediction (As)')
plt.scatter(longitude, latitude, c='white', edgecolor='black', label='Data Points')
plt.title('Global Linear Plane Prediction of Arsenic Concentration')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.show()

In [ ]:
# ------------------------------
# 3. Compare Models (RMSE)
# ------------------------------
# Kriging predictions at original data points
arsenic_kriging_pred, _ = OK.execute('points', longitude, latitude)
rmse_kriging = np.sqrt(mean_squared_error(arsenic, arsenic_kriging_pred))


In [ ]:
# Global linear model predictions at original data points
arsenic_global_pred_points = linear_model.predict(coords)
rmse_global = np.sqrt(mean_squared_error(arsenic, arsenic_global_pred_points))

In [ ]:
print(f"RMSE for Kriging: {rmse_kriging:.3f}")
print(f"RMSE for Global Model: {rmse_global:.3f}")

In [ ]:
# ------------------------------
# 4. Suggest New Sampling Locations
# ------------------------------
# Identify regions with high standard error for additional sampling
plt.figure(figsize=(10, 6))
plt.contourf(gridx, gridy, arsenic_std, cmap='plasma')
plt.colorbar(label='Standard Error')
plt.scatter(longitude, latitude, c='white', edgecolor='black', label='Existing Points')

In [ ]:
# Mark suggested new sampling locations in regions with high standard error
new_x = [gridx[25], gridx[75]]
new_y = [gridy[25], gridy[75]]
plt.scatter(new_x, new_y, c='red', marker='x', s=100, label='New Sampling Points')

In [ ]:
plt.title('Suggested New Sampling Locations')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.show()